In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, UpSampling2D, BatchNormalization
from keras.optimizers import Adam

# Define the generator model
def create_generator():
    model = Sequential()

    model.add(Dense(256*8*8, activation="relu", input_dim=100))
    model.add(Reshape((8, 8, 256)))

    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
   
    model.add(Conv2D(3, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    return model

# Define the discriminator model
def create_discriminator():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(64,64,3), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

# Now that we have our generator and discriminator, we can set up the GAN
def create_gan(discriminator, generator):
    model = Sequential()

    # Combined model
    model.add(generator)
    model.add(discriminator)

    return model

# Create and compile the discriminator
discriminator = create_discriminator()
discriminator.compile(loss='binary_crossentropy', 
                      optimizer=Adam(0.0002, 0.5), 
                      metrics=['accuracy'])

# Create and compile the generator
generator = create_generator()
generator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Create and compile the combined network
gan = create_gan(discriminator, generator)
gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
